In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# Model 248 is for canceling error
# Model 14/167 for a type three error
simple = load_file_from_curated_data(87)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

Capping: Cdc13 + Utelo -> Ctelo
Uncapping: Ctelo -> Cdc13 + Utelo
Rad17binding: Utelo + Rad17 + Rad24 + ATP -> Rad17Utelo + Rad24 + ADP
ExoXactivation: ExoXI + Rad17Utelo -> ExoXA + Rad17Utelo
ExoXactivity: ExoXA + Rad17Utelo -> ExoXA + Rad17Utelo + ssDNA
Exo1activation: Exo1I -> Exo1A
Exo1Rad24dependentActivation: Exo1I + Rad24 -> Exo1A + Rad24
Exo1Rad17independentActivity: Utelo + Exo1A -> Utelo + Exo1A + ssDNA
Exo1Rad17dependentActivity: Rad17Utelo + Exo1A -> Rad17Utelo + Exo1A + ssDNA
RPAbinding1: ssDNA + RPA -> RPAssDNA1
RPAbinding2: ssDNA + RPAssDNA1 -> RPAssDNA2
RPAbinding: ssDNA + RPAssDNA2 -> RPAssDNA
Mec1binding: RPAssDNA + Mec1 -> Mec1RPAssDNA
Rad9activation: Rad9Kin + Rad9I -> Rad9Kin + Rad9A
ExoXinhibition: ExoXA + Rad9A -> ExoXI + Rad9A
ExoXinhibition2: ExoXA + Rad9I -> ExoXI + Rad9I
Rad53activation: Rad53I + Rad9A -> Rad53A + Rad9A
Chk1Activation: Chk1I + Rad9A -> Chk1A + Rad9A
Exo1inhibition: Exo1A + Rad53A -> Exo1I + Rad53A
Dun1activation: Dun1I + Rad53A -> Dun1A + Rad

In [3]:
canceling_report = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)

We detected a mass imbalance from the following reactions:

1. ExoXactivity: ExoXA + Rad17Utelo -> ExoXA + Rad17Utelo + ssDNA

*ExoXA is a common molecule in reactants and products, so can be canceled

*Rad17Utelo is a common molecule in reactants and products, so can be canceled


Therefore, they will result in empty reactant with zero mass:

ExoXactivity:  -> {ssDNA}

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

----------------------------------------------------------------------

We detected a mass imbalance from the following reactions:

1. Exo1Rad17independentActivity: Utelo + Exo1A -> Utelo + Exo1A + ssDNA

*Utelo is a common molecule in reactants and products, so can be canceled

2. Exo1activation: Exo1I -> Exo1A

*Exo1I and Exo1A have the same mass according to the above reaction


Therefore, they will result in empty reactant with zero mass:

Exo1Rad17independentActivity:  -> {ssDNA}

This indicate

In [4]:
echelon_report = gr.reportEchelonError(m.echelon_errors)
print(echelon_report)
detailed_echelon_report = gr.reportEchelonError(m.echelon_errors, explain_details=True)
#print(detailed_echelon_report)

In [5]:
type3_report = gr.reportTypeThreeError(m.type_three_errors)
#print(type3_report)
detailed_type3_report = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
#print(detailed_type3_report)

IndexError: list index out of range

In [ ]:
m.type_one_som_errors

In [8]:
"%.2f" % (3.00)

'3.00'

In [6]:
m.rref_df

,PstatDimerisation,statPhosphorylation,PstatDimerisationNuc
{Pstat_sol},-2.0,0.0,0.0
{species_test},0.0,1.0,0.0
"{PstatDimer_nuc, PstatDimer_sol}",0.0,0.0,1.0
"{Pstat_nuc, stat_nuc, stat_sol}",2.0,0.0,-2.0


In [12]:
molecule1 = simple.getMolecule("stat_sol")
molecule2 = simple.getMolecule("Pstat_nuc")
som = m.getNode(stat_sol)
#gr.getSOMPath(som, stat_sol, Pstat_nuc)
print(som)
for r in som.reactions:
  print(r.makeIdentifier(is_include_kinetics=False))

{Pstat_nuc, stat_nuc, stat_sol}
statDephosphorylation: Pstat_nuc -> stat_nuc
stat_export: stat_sol -> stat_nuc


In [9]:
print(m.convertReactionToSOMReaction(simple.getReaction("statPhosphorylation")))
print(m.convertReactionToSOMReaction(simple.getReaction("PstatDimerisation")))
print(m.convertReactionToSOMReaction(simple.getReaction("PstatDimerisationNuc")))

statPhosphorylation: {Pstat_nuc, stat_nuc, stat_sol} -> {species_test} + {Pstat_sol}
PstatDimerisation: 2.00 {Pstat_sol} -> {PstatDimer_nuc, PstatDimer_sol}
PstatDimerisationNuc: 2.00 {Pstat_nuc, stat_nuc, stat_sol} -> {PstatDimer_nuc, PstatDimer_sol}


In [32]:
print(list(soms)[0].reactions)

{PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm}


In [27]:
def getMoleculeLinkage(self, som_name, reactions):
  """
  Create two lists. 
  1. molecules in the reactions that are in the same som
  2. reactions used to merge the molecules
  :param str som_name:
  :param list-str reactions:
  :return list-str: linked_molecules
  :return list-str: linked_reactions
  """
  som = self.findSOM(som_name)
  molecules = {m.name for m in som.molecules}
  # linked_molecules: molecules within both the SOM and given reactions
  linked_molecules = set()
  for r in reactions:
    reaction = self.mesgraph.simple.getReaction(r)
    reactants = {m.molecule.name for m in reaction.reactants}
    products = {m.molecule.name for m in reaction.products}
    reaction_molecules = reactants.union(products)
    #
    som_molecules = molecules.intersection(reaction_molecules)
    linked_molecules = linked_molecules.union(som_molecules)
  linked_reactions = []
  for sr in som.reactions:
    sreactants = {m.molecule.name for m in sr.reactants}
    sproducts = {m.molecule.name for m in sr.products}
    if sreactants.intersection(linked_molecules) and \
        sproducts.intersection(linked_molecules):
      linked_reactions.append(sr.label)
  return list(linked_molecules), linked_reactions

In [18]:
reactions = [ro.reaction for ro in op]
som = m.getNode(simple.getMolecule("Pstat_sol"))

a, b = gr.getMoleculeLinkage(som.identifier,reactions)
print(a)
print(b)

['Pstat_sol']
[]


In [12]:
# this will test which model has which error;;
for i in range(1, 704):
  try:
    simple = load_file_from_curated_data(i)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.canceling_errors:
      print("model", i, " has canceling error!")
      break
  except:
    pass

model 87  has canceling error!


In [47]:
m.type_one_errors

[]

In [4]:
gr = GAMESReport(m)
print(gr.reportTypeOneError(m.type_one_errors))


FH4 = CH2FH4 by reaction(s):
SHMTr: CH2FH4 -> FH4

However, FH4 < CH2FH4 by reaction(s):
CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO
SHMT: FH4 + serine -> CH2FH4
HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4
**************************************************

CH2FH4 = FH4 by reaction(s):
SHMTr: CH2FH4 -> FH4

FH4 = FH2f by reaction(s):
DHFReductase: FH2f -> FH4

However, CH2FH4 < FH2f by reaction(s):
TYMS: CH2FH4 + dUMP -> FH2f
**************************************************

FFH2 = FH2f by reaction(s):
FFH2syn: FH2f -> FFH2

However, FFH2 < FH2f by reaction(s):
ATIC12: FFH2 + AICAR -> FH2f
**************************************************

MTX1 = MTX1b by reaction(s):
MTX1deg: MTX1b -> MTX1

However, MTX1 < MTX1b by reaction(s):
MTX1off: MTX1b -> MTX1 + DHFRf
MTX1on: MTX1 + DHFRf -> MTX1b
**************************************************

MTX2 = MTX2b by reaction(s):
MTX2deg: MTX2b -> MTX2

However, MTX2 < MTX2b by reaction(s):
MTX2off: MTX2b -> MTX2 + DHFRf
MTX2on: MTX2 + DHFRf -> MTX2b
**********

In [5]:
m.type_two_errors

[[{CHOFH4}, {CH2FH4, FFH2, FH2f, FH4}], [{CH3FH4}, {CH2FH4, FFH2, FH2f, FH4}]]

In [6]:
type_two_report = gr.reportTypeTwoError(m.type_two_errors)
print(type_two_report)


CHOFH4 < FH4 by
ATIC7: CHOFH4 + AICAR -> FH4

FH4 < CHOFH4 by
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4

CH2FH4 < CHOFH4 by
MTHFD: CH2FH4 + NADP -> CHOFH4

FH4 = CH2FH4 by
SHMTr: CH2FH4 -> FH4
**************************************************


CH3FH4 < FH4 by
MTR: CH3FH4 + homocysteine -> FH4

CH2FH4 < CH3FH4 by
MTHFR: CH2FH4 + NADPH -> CH3FH4

FH4 = CH2FH4 by
SHMTr: CH2FH4 -> FH4
**************************************************


--------------------------------------------------



In [8]:
echelon_report = gr.reportEchelonError(m.echelon_errors)
print(echelon_report)


The following reactions create a mass imbalance:

HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4
ATIC7: CHOFH4 + AICAR -> FH4

->->->->->->->->->->->->->->->->->->->->->->->->->
FH4 is a common element in the reactions above.
<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-

->->->->->->->->->->->->->->->->->->->->->->->->->
CHOFH4 is a common element in the reactions above.
<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-

**************************************************

The following reactions create a mass imbalance:

MTHFD: CH2FH4 + NADP -> CHOFH4
ATIC7: CHOFH4 + AICAR -> FH4

->->->->->->->->->->->->->->->->->->->->->->->->->
The following molecules,
CH2FH4
FH4
Have equal mass by the following reaction(s).
SHMTr: CH2FH4 -> FH4
<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-

->->->->->->->->->->->->->->->->->->->->->->->->->
CHOFH4 is a common element in the reactions above.
<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-

**********************************************

In [9]:
error_cycle = []
cycle = m.type_two_errors[0]
cycle2 = cycle[1:] + [cycle[0]]
for first, second in zip(cycle, cycle2):
  print("pairs:")
  print(first, second)
  som1_moles = {m.name for m in list(first.molecules)}
  som2_moles = {m.name for m in list(second.molecules)}
  reaction_data = m.get_edge_data(first, second)[cn.REACTION]
  nodes1 = []
  nodes2 = []
  reaction_labels = []  
  for r in reaction_data:
    reaction = simple.getReaction(r)
    print(reaction.makeIdentifier(is_include_kinetics=False))  
    if reaction.category == cn.REACTION_n_1:
      sources = {r.molecule.name for r in reaction.reactants}
      destinations = {p.molecule.name for p in reaction.products}
    elif reaction.category == cn.REACTION_1_n:
      sources = {p.molecule.name for p in reaction.products}
      destinations = {r.molecule.name for r in reaction.reactants}
    node2 = list(destinations.intersection(som2_moles))[0]
    for node1 in list(sources.intersection(som1_moles)):
      nodes1.append(node1)
      nodes2.append(node2)
      reaction_labels.append(reaction.label)
  error_cycle.append(cn.PathComponents(node1=nodes1, 
                                     node2=nodes2,
                                     reactions=reaction_labels))
error_cycle

pairs:
{CHOFH4} {CH2FH4, FFH2, FH2f, FH4}
ATIC7: CHOFH4 + AICAR -> FH4
pairs:
{CH2FH4, FFH2, FH2f, FH4} {CHOFH4}
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4
MTHFD: CH2FH4 + NADP -> CHOFH4


[PathComponents(node1=['CHOFH4'], node2=['FH4'], reactions=['ATIC7']),
 PathComponents(node1=['FH4', 'CH2FH4'], node2=['CHOFH4', 'CHOFH4'], reactions=['HCOOHtoCHOFH4', 'MTHFD'])]

In [13]:
  def getSOMPath(som, molecule1, molecule2):
    """
    Create an undirected graph between
    two molecules within a SOM
    and find the shortest path
    :param SOM som:
    :param str mole1:
    :param str mole2:
    :return list-PathComponents som_path:
    """   
#     molecule1 = mole1.name
#     molecule2 = mole2.name
    # construct undirected graph
    subg = nx.Graph()
    # here, every reaction is 1-1 reaction
    for reaction in list(som.reactions):
      node1 = reaction.reactants[0].molecule.name
      node2 = reaction.products[0].molecule.name
      if subg.has_edge(node1, node2):
        reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
        # if reaction.label is not already included in the attribute,
        if reaction.label not in set(reaction_label):
          reaction_label = reaction_label + [reaction.label]
      else:
        reaction_label = [reaction.label]    
      subg.add_edge(node1, node2, reaction=reaction_label)
    path = [short_p for short_p in nx.shortest_path(subg, 
                                                    source=molecule1, 
                                                    target=molecule2)]
    som_path = []
    for idx in range(len(path)-1):
      edge_reactions = subg.get_edge_data(path[idx], path[idx+1])[cn.REACTION]
      som_path.append(cn.PathComponents(node1=path[idx], 
                                     node2=path[idx+1],
                                     reactions=edge_reactions))
    return som_path

In [15]:
error_cycle

[PathComponents(node1=['CHOFH4'], node2=['FH4'], reactions=['ATIC7']),
 PathComponents(node1=['FH4', 'CH2FH4'], node2=['CHOFH4', 'CHOFH4'], reactions=['HCOOHtoCHOFH4', 'MTHFD'])]

In [26]:
set(error_cycle[0].node2).intersection(set(error_cycle[1].node1))

{'FH4'}

In [32]:
last_node = None
for one_path in error_cycle:
  comb = zip(one_path.node1, one_path.node2, one_path.reactions)
  if last_node is not None:
    common_molecules = last_node.intersection(set(one_path.node1))
    if common_molecules:
      print("**Already havce common molecules", common_molecules)
    else:
      print("**need to find path between: ", last_node, one_path.node1)
  last_node = set(one_path.node2)
  for tup in comb:
    print(tup[0], cn.LESSTHAN, tup[1], "by", tup[2])
    print(simple.getReaction(tup[2]).makeIdentifier(is_include_kinetics=False))
    print("")
  if len(one_path.node1) > 1:
    for node1, node2 in zip(one_path.node1, one_path.node1[1:]):
      print("node pairs whose equality needs to be examined")
      print(node1, node2)
      mole1 = simple.getMolecule(node1)
      mole2 = simple.getMolecule(node2)
      som = m.getNode(mole1)
      som_path = getSOMPath(som, node1, node2)
      print("som_path is: ", )
      print(som_path)

CHOFH4 < FH4 by ATIC7
ATIC7: CHOFH4 + AICAR -> FH4

**Already havce common molecules {'FH4'}
FH4 < CHOFH4 by HCOOHtoCHOFH4
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4

CH2FH4 < CHOFH4 by MTHFD
MTHFD: CH2FH4 + NADP -> CHOFH4

node pairs whose equality needs to be examined
FH4 CH2FH4
som_path is: 
[PathComponents(node1='FH4', node2='CH2FH4', reactions=['SHMTr'])]


In [77]:
list(zip(error_cycle[0].node1, error_cycle[0].node2, error_cycle[0].reactions))

[('FH4', 'CHOFH4', 'HCOOHtoCHOFH4'), ('CH2FH4', 'CHOFH4', 'MTHFD')]

In [41]:
cycle[1:] + [cycle[0]]

[{CH2FH4, FFH2, FH2f, FH4}, {CHOFH4}]

In [9]:
gr.mesgraph.som_stoichiometry_matrix.index

Index(['{homocysteine}', '{formate}', '{NADPH}', '{GAR}', '{ATP}', '{CH3FH4}',
       '{DHFRf}', '{HCHO}', '{NADP}', '{dUMP}', '{serine}', '{CHOFH4}',
       '{CH2FH4, FFH2, FH2f, FH4}', '{AICAR, FGAR}',
       '{EMTX, MTX1, MTX1b, MTX2, MTX2b, MTX3, MTX3b, MTX4, MTX4b, MTX5, MTX5b}'],
      dtype='object')

In [7]:
print(m.convertReactionToSOMReaction(simple.getReaction("HCOOHtoCHOFH4")))
print(m.convertReactionToSOMReaction(simple.getReaction("ATIC7")))

HCOOHtoCHOFH4: {formate} + {ATP} + {CH2FH4, FFH2, FH2f, FH4} -> {CHOFH4}
ATIC7: {CHOFH4} + {AICAR, FGAR} -> {CH2FH4, FFH2, FH2f, FH4}


In [9]:
a, b = gr.getMoleculeLinkage("{CHOFH4}", ["ATIC7"])
print(a)
print(b)

{'CHOFH4'}
[]


In [13]:
a, b = gr.getMoleculeLinkage("{CH2FH4, FFH2, FH2f, FH4}", ["ATIC7", "HCOOHtoCHOFH4"])
print(a)
print(b)

{'FH4'}
[]


In [21]:
echelon_error = m.echelon_errors[0]
echelon_error

MTHFD: {NADP} + {AICAR, FGAR} -> 

In [124]:
result_row = m.echelon_df[echelon_error.label]
nonzero_result_row = result_row[result_row.to_numpy().nonzero()[0]]
nonzero_result_row.index

Index(['{NADP}', '{AICAR, FGAR}'], dtype='object')

In [41]:
if m.rref_df is None:
  operation_df = m.lower_inverse
else:
  operation_df = m.rref_operation.dot(m.lower_inverse)
operation_df
op_row = operation_df.T[echelon_error.label]
nonzero_op_rows = op_row[op_row.to_numpy().nonzero()[0]]
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [49]:
for r in nonzero_op_rows.index:
  reaction = simple.getReaction(r)
  print(reaction.makeIdentifier(is_include_kinetics=False))
  print(m.convertReactionToSOMReaction(reaction))
  print()

ATIC7: CHOFH4 + AICAR -> FH4
ATIC7: {CHOFH4} + {AICAR, FGAR} -> {CH2FH4, FFH2, FH2f, FH4}

MTHFD: CH2FH4 + NADP -> CHOFH4
MTHFD: {NADP} + {CH2FH4, FFH2, FH2f, FH4} -> {CHOFH4}



In [55]:
# find out canceled reactants/products first
# meaning, nonzeros that both exist in atic7 and mthfd but not in the result_row
atic7_row = m.som_stoichiometry_matrix[nonzero_op_rows.index[0]]
mthfd_row = m.som_stoichiometry_matrix[nonzero_op_rows.index[1]]
nonzero_atic7_row = atic7_row[atic7_row.to_numpy().nonzero()[0]]
nonzero_mthfd_row = mthfd_row[mthfd_row.to_numpy().nonzero()[0]]
print(nonzero_atic7_row.index)
print(nonzero_mthfd_row.index)

Index(['{CHOFH4}', '{CH2FH4, FFH2, FH2f, FH4}', '{AICAR, FGAR}'], dtype='object')
Index(['{CHOFH4}', '{NADP}', '{CH2FH4, FFH2, FH2f, FH4}'], dtype='object')


In [57]:
nonzero_atic7_row.index.intersection(nonzero_mthfd_row.index)

Index(['{CHOFH4}', '{CH2FH4, FFH2, FH2f, FH4}'], dtype='object')

In [123]:
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [63]:
nonzero_elements = m.som_stoichiometry_matrix.index
for r in nonzero_op_rows.index:
  row = m.som_stoichiometry_matrix[r]
  nonzero_elements = nonzero_elements.intersection(row[row.to_numpy().nonzero()[0]].index)
# finally, subtract remaining nonzero elements from nonzero_result_row
canceled_elements = nonzero_elements.difference(nonzero_result_row.index)
print(canceled_elements)

Index(['{CH2FH4, FFH2, FH2f, FH4}', '{CHOFH4}'], dtype='object')


In [73]:
def getSOM(som_name):
  for node in m.nodes:
    if node.identifier == som_name:
      return node

{CH2FH4, FFH2, FH2f, FH4}

In [77]:
som1 = getSOM(canceled_elements[0])
som2 = getSOM(canceled_elements[1])

{CHOFH4}

In [125]:
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [121]:
# this is for each som that were eliminated by matrix operation
molecules = {m.name for m in som1.molecules}
linked_molecules = set()
for r in nonzero_op_rows.index:
  reaction = simple.getReaction(r)
  reactants = {m.molecule.name for m in reaction.reactants}
  products = {m.molecule.name for m in reaction.products}
  som_molecules = molecules.intersection(reactants)
  som_molecules = som_molecules.union(molecules.intersection(products))
  linked_molecules = linked_molecules.union(som_molecules)
# now need to path (reactions) between molecules within the linked_molecules
linked_reactions = []
for sr in som1.reactions:
  sreactants = {m.molecule.name for m in sr.reactants}
  sproducts = {m.molecule.name for m in sr.products}
  if sreactants.intersection(linked_molecules) and \
     sproducts.intersection(linked_molecules):
    linked_reactions.append(sr.label)
# can generate report that will create equality between molecules -> maximum amount of information

In [122]:
print(linked_molecules)
print(linked_reactions)

{'CH2FH4', 'FH4'}
['SHMTr']


In [113]:
som1.reactions

{DHFReductase: FH2f -> FH4; cell * kter * FH2b,
 FFH2syn: FH2f -> FFH2; cell * Vm * FH2f,
 SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1))}

In [120]:
sreactants

{'FH2f'}